In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
pd.options.mode.chained_assignment = None
%matplotlib inline

In [14]:
import dgl
from dgl.data import DGLDataset
import torch
import os

edges_fMRI = pd.read_csv('./BNU1_fMRI/fMRI_brain_graph_weighted_edges_0.8.csv')
# edges_dMRI_fused = pd.read_csv('./BNU1_dMRI/dMRI_brain_graph_edges_fused.csv')
# edges_dMRI_ses1 = pd.read_csv('./BNU1_dMRI/dMRI_brain_graph_edges_ses1.csv')
# edges_dMRI_ses2 = pd.read_csv('./BNU1_dMRI/dMRI_brain_graph_edges_ses2.csv')
properties = pd.read_csv('./BNU1_fMRI/fMRI_brain_graph_properties.csv')

edges_fMRI.head()

properties.head()

class BNU1Dataset(DGLDataset):
    def __init__(self):
        super().__init__(name='synthetic')

    def process(self):
        edges = pd.read_csv('./BNU1_fMRI/fMRI_brain_graph_weighted_edges_0.8.csv')
        properties = pd.read_csv('./BNU1_fMRI/fMRI_brain_graph_properties.csv')
        self.graphs = []
        self.labels = []

        # Create a graph for each graph ID from the edges table.
        # First process the properties table into two dictionaries with graph IDs as keys.
        # The label and number of nodes are values.
        label_dict = {}
        num_nodes_dict = {}
        for _, row in properties.iterrows():
            label_dict[row['graph_id']] = row['label']
            num_nodes_dict[row['graph_id']] = row['num_nodes']

        # For the edges, first group the table by graph IDs.
        edges_group = edges.groupby('graph_id')

        # For each graph ID...
        for graph_id in edges_group.groups:
            # Find the edges as well as the number of nodes and its label.
            edges_of_id = edges_group.get_group(graph_id)
            src = edges_of_id['src'].to_numpy()
            dst = edges_of_id['dst'].to_numpy()
            weight = edges_of_id['weight'].to_numpy()
            num_nodes = num_nodes_dict[graph_id]
            label = label_dict[graph_id]

            # Create a graph and add it to the list of graphs and labels.
            g = dgl.graph((src, dst), num_nodes=num_nodes)
            weight = torch.FloatTensor(weight)
            g.edata['weight'] = weight
            self.graphs.append(g)
            self.labels.append(label)

        # Convert the label list to tensor for saving.
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

dataset = BNU1Dataset()
graph, label = dataset[0]
print(graph, label)

Graph(num_nodes=70, num_edges=24,
      ndata_schemes={}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float32)}) tensor(1)


In [45]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import re
%matplotlib inline

graph_idx = 0
pos_idx = []
neg_idx = []
f = csv.reader(open('./BNU1_fMRI/BNU1_phenotypic_data.csv','r')) 
for j, row in enumerate(f):
    if j != 0 and j % 2 ==1:
        label = row[3]
        filter(str.isdigit, label)
        if int(label)-1 == 1:
            pos_idx.append(graph_idx)
        else:
            neg_idx.append(graph_idx)
        graph_idx += 1

edge_list_2_downsampling = []
edge_list_2_property = [] 
pos_adj_list = None
neg_adj_list = None
length = 25920 - 25864 + 1
for i in range(length):
    G00 = nx.read_gpickle("./BNU1_fMRI/sub-00%s_ses-1_bold_desikan_res-2x2x2_measure-correlation.gpickle"%(int(i+25864)))
    edge_list_2 = []
    edge = G00.adj
    for src in edge:
        for dst in edge[src]:
            edge_list_2.append([src, dst, edge[src][dst]['weight']])
    for edge in edge_list_2:
        if edge[0] == edge[1]:
            continue
        elif edge[2] <= 0.8:
            continue
        else:
            edge_list_2_downsampling.append([edge[0]-1, edge[1]-1, edge[2]])
    G2 = nx.Graph()
    G2.add_weighted_edges_from(edge_list_2_downsampling)
    A2 = nx.to_numpy_array(G2)
    if i in pos_idx and type(pos_adj_list) == None:
        pos_adj_list = A2
    elif i in neg_idx and type(neg_adj_list) == None:
        neg_adj_list = A2
    else:
        print(np.shape(A2))
        if i in pos_idx:
            np.stack((pos_adj_list, A2),axis=0)
        else:
            np.stack((neg_adj_list, A2),axis=0)
# pos_adj_list=np.array(pos_adj_list)
# neg_adj_list=np.array(neg_adj_list, dtype=object)
print(pos_adj_list)
# a
#     G2 = nx.Graph()
#     G2.add_weighted_edges_from(edge_list_2)
        
# df = pd.DataFrame([iter for iter in edge_list_2_downsampling], columns=['graph_id', 'src', 'dst', 'weight'])
# df.to_csv('./BNU1_fMRI/fMRI_brain_graph_weighted_edges_0.8.csv', index=False)       
# df = pd.DataFrame([iter for iter in edge_list_2_property], columns=['graph_id', 'label', 'num_nodes'])
# df.to_csv('./BNU1_fMRI/fMRI_brain_graph_properties.csv', index=False)
# print('Finished preprocessing weighted fMRI graph!')

(17, 17)


ValueError: all input arrays must have the same shape

In [2]:
from graspologic.datasets import load_mice

# Load the full mouse dataset
mice = load_mice()

# Stack all adjacency matrices in a 3D numpy array
graphs = np.array(mice.graphs)

# Get sample parameters
n_subjects = mice.meta["n_subjects"]
n_vertices = mice.meta["n_vertices"]

# Split the set of graphs by genotype
btbr = graphs[mice.labels == "BTBR"]
b6 = graphs[mice.labels == "B6"]
cast = graphs[mice.labels == "CAST"]
dba2 = graphs[mice.labels == "DBA2"]

connectomes = [btbr, b6, cast, dba2]

[[[0.000e+00 3.009e+03 1.914e+03 ... 0.000e+00 1.000e+01 4.200e+01]
  [3.009e+03 0.000e+00 1.936e+03 ... 0.000e+00 0.000e+00 1.900e+01]
  [1.914e+03 1.936e+03 0.000e+00 ... 0.000e+00 0.000e+00 2.000e+00]
  ...
  [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 6.000e+00 2.000e+00]
  [1.000e+01 0.000e+00 0.000e+00 ... 6.000e+00 0.000e+00 2.300e+01]
  [4.200e+01 1.900e+01 2.000e+00 ... 2.000e+00 2.300e+01 0.000e+00]]

 [[0.000e+00 4.762e+03 7.770e+02 ... 1.000e+00 1.000e+00 2.480e+02]
  [4.762e+03 0.000e+00 8.580e+02 ... 0.000e+00 0.000e+00 2.000e+00]
  [7.770e+02 8.580e+02 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  ...
  [1.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 3.000e+00 2.200e+01]
  [1.000e+00 0.000e+00 0.000e+00 ... 3.000e+00 0.000e+00 1.000e+00]
  [2.480e+02 2.000e+00 0.000e+00 ... 2.200e+01 1.000e+00 0.000e+00]]

 [[0.000e+00 3.279e+03 9.970e+02 ... 0.000e+00 0.000e+00 6.800e+01]
  [3.279e+03 0.000e+00 1.508e+03 ... 0.000e+00 1.000e+00 5.000e+00]
  [9.970e+02 1.508e+03 0.000e+00

In [20]:
print(type(btbr[0]))

<class 'numpy.ndarray'>


In [35]:
# Embed mice graphs
# Single graph embedding
from graspologic.embed import node2vec_embed, AdjacencySpectralEmbed, LaplacianSpectralEmbed
# Multiple graph embedding
from graspologic.embed import OmnibusEmbed, MultipleASE, mug2vec
from graspologic.plot import pairplot

# Jointly embed graphs using omnibus embedding
embedder = OmnibusEmbed()
omni_embedding = embedder.fit_transform(pos_adj_list)
print(omni_embedding.shape)

ValueError: Input tensor must be 3-dimensional, not 1-dimensional.